In [1]:
import pandas as pd
train_df = pd.read_csv('../data/raw/soqg/train_chunk_I.csv')
print(train_df.head())
print(train_df.columns)

   Unnamed: 0                                              input  \
0           0  alternate_viewpoints_perspectives: A parallel ...   
1           1  alternate_viewpoints_perspectives: It would be...   
2           2  alternate_viewpoints_perspectives: I do not un...   
3           3  alternate_viewpoints_perspectives: There is a ...   
4           4  alternate_viewpoints_perspectives: You conside...   

                                              target  
0               What about nations who have nothing?  
1         If not, what about this is cringe exactly?  
2            What about public surveillance cameras?  
3  How about allowing some students to go straigh...  
4  What else do you imagine is necessary to be co...  
Index(['Unnamed: 0', 'input', 'target'], dtype='object')
